## Course: TM10007 - Machine learning
Editors: Lishia Vergeer, Amy Roos, Maaike Pruijt, Hilde Roording.

Description: The aim of this code is to predict the tumor grade of glioma’s(high or low) before surgery, 
based on features extracted from a combination of four MRI images: 
T2-weighted, T2-weighted FLAIR and T1-weighted before and after injection of contrast agent.

#### Import packages

In [152]:
# General packages
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import datasets as ds

# Import code
from brats.load_data import load_data
from sklearn.model_selection import train_test_split
from sklearn import decomposition

# scaler
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler

# Classifiers
from sklearn.decomposition import PCA
from sklearn import metrics
from sklearn import feature_selection 
from sklearn import preprocessing
from sklearn import neighbors
from sklearn import svm

#### Load data

In [153]:
# Data loading functions. Uncomment the one you want to use
from brats.load_data import load_data

data_brats = load_data()
print(f'The number of samples in data_brats: {len(X.index)}')
print(f'The number of columns in data_brats: {len(X.columns)}')

# Convert to dataframe
X = pd.DataFrame(data_brats)


The number of samples in data_brats: 167
The number of columns in data_brats: 724


c:\Users\maaik\Desktop\TM10007_Machine_learning\TM10007_ML\TM10007\brats\load_data.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append(data2)


#### Split data in X and y
Split in X (data) and y (label)


- Checken of splitten van data en labels inderdaad moet na het er uithalen van NaN's --> Nu ervoor!

In [154]:
# split column label from dataset X
y = X.pop('label')
print(f'The number of samples in y: {len(y.index)}')


The number of samples in y: 167


#### Split data
This function creates a panda dataframe and splits the data into test and train components.
This is done with test_size variable and the function train_test_split from the sklearn module.
Returns a train set with the data of 80% and a test set of 20% of the subjects.



In [155]:
# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.1)  

#### No None
Deleting columns with NaN or filling them.
- Bepalen waar threshold ligt

In [156]:
# Insight in the data
print(f'OVERZICHT: {X_train.isnull().sum()}')

# infinity to NaN
X_train[X_train==np.inf]=np.nan

# non-numeric features to NaN
X_train = X_train.replace(['#DIV/0!'], np.nan)
X_train = X_train.apply(pd.to_numeric, errors='coerce')

# If the total number of NaN observations in a column are greater than 40%, delete the entire column.
perc = 40.0
min_count = int(((100-perc)/100)*X_train.shape[0] + 1)
data_dropcolumn = X_train.dropna(axis=1, thresh=min_count)

# fill the NaN observations.
data_fill = data_dropcolumn.fillna(data_dropcolumn.median()) #Bekijken mean of median
# print(data_fill)

# Inzicht in data
print(f'OVERZICHT NONONE: {data_fill.isnull().sum()}')


OVERZICHT: VOLUME_ET        0
VOLUME_NET       0
VOLUME_ED        0
VOLUME_TC        0
VOLUME_WT        0
              ... 
TGM_T_5        147
TGM_Cog_X_6    149
TGM_Cog_Y_6    149
TGM_Cog_Z_6    149
TGM_T_6        149
Length: 724, dtype: int64
OVERZICHT NONONE: VOLUME_ET      0
VOLUME_NET     0
VOLUME_ED      0
VOLUME_TC      0
VOLUME_WT      0
              ..
TGM_dw         0
TGM_Cog_X_1    0
TGM_Cog_Y_1    0
TGM_Cog_Z_1    0
TGM_T_1        0
Length: 704, dtype: int64


#### Scale features

In [157]:
# robustscaler
scaler_three = RobustScaler()
scaler_three.fit(data_fill)
X_scaled = scaler_three.transform(data_fill)

print(X_scaled)

[[ 0.44261689 -0.3793774   0.26399166 ... -0.49758091  0.2301565
  -0.10666034]
 [-0.44620939  0.61869312 -0.12028    ... -0.09553132 -0.88209699
  -0.27184741]
 [ 0.15056793  1.13852722  0.44383111 ... -0.94431519  0.31676419
   2.73666138]
 ...
 [ 2.35593907  0.33094301  0.08354311 ...  0.43424679 -0.16555941
   2.14296643]
 [ 0.07149775 -0.12362316 -0.67613006 ...  0.80001105 -0.18177622
   0.02097008]
 [ 0.57726512 -0.33465004  0.44527774 ... -0.29857362 -0.99049495
  -0.05765003]]


#### Transform features
- We denken alleen PCA te gebruiken. Klopt het dat je dan niet ook selectie gebruikt?
- PCA gaat uit van lineair model. Hoe kunnen we weten of ons onze data daar geschikt voor is?
- Is het de bedoeling dat we ons hier verder in verdiepen of valt dat buiten de scope van het vak?
- Uitzoeken hoe we de X_test en y_test correct gereed krijgen voor PCA.

In [158]:
# Perform a PCA
pca = decomposition.PCA(n_components=2)
pca.fit(X_scaled) 
X_train_pca = pca.transform(X_scaled)
print(X_train_pca)

#X_test_pca = pca.transform(X_test)


[[ 1.14377304e+01 -2.75923877e+02]
 [ 1.14048696e+01 -2.75892201e+02]
 [ 1.12620555e+01 -2.75840009e+02]
 [ 1.13658753e+01 -2.75868717e+02]
 [ 1.14776062e+01 -2.76005801e+02]
 [ 1.13485175e+01 -2.75830674e+02]
 [-2.78827635e+03  3.05535467e+03]
 [ 1.13543161e+01 -2.75920046e+02]
 [ 1.13270582e+01 -2.75959192e+02]
 [ 3.75090920e+03  2.99610088e+03]
 [ 1.13942098e+01 -2.75898348e+02]
 [ 1.14046637e+01 -2.75893694e+02]
 [ 1.17726060e+01 -2.75617805e+02]
 [ 1.12595217e+01 -2.75928615e+02]
 [ 7.10656116e+00 -2.70997473e+02]
 [ 1.15661743e+01 -2.76036390e+02]
 [ 1.14503311e+01 -2.75974876e+02]
 [ 1.13315658e+01 -2.75975379e+02]
 [ 1.14124269e+01 -2.75918326e+02]
 [ 1.13739748e+01 -2.75891144e+02]
 [-7.71182135e+02  6.19361506e+02]
 [ 3.59389853e+01 -2.54481006e+02]
 [ 1.13515493e+01 -2.75914045e+02]
 [ 8.14350579e+02  4.27143295e+02]
 [ 1.30575771e+01 -2.74496617e+02]
 [ 1.13655163e+01 -2.75948942e+02]
 [ 1.13942154e+01 -2.75911591e+02]
 [ 1.14523135e+01 -2.75937485e+02]
 [ 7.57393214e+01  3

#### Classifier: kNN

In [159]:
# Fit kNN
knn = neighbors.KNeighborsClassifier(n_neighbors=15)
knn.fit(X_train_pca, y_train)
score_train = knn.score(X_train_pca, y_train)
#score_test = knn.score(X_test_pca, y_test)

# Print result
print(f"Training result: {score_train}")
#print(f"Test result: {score_test}")

Training result: 0.6866666666666666


#### Classifier: SVM

#### Classifier: Random Forest